In [1]:
%matplotlib inline
import copy
import qutip
from qutip import Qobj
import numpy as np
import math, cmath
import matplotlib.pyplot as plt
from tqdm import tqdm, trange

from stateobj import Physics
import utilities as use
from entropy import entropy_vn
import circuits as qc

import os

from IPython.display import Markdown, display

In [2]:
TIMESTEPS = 1500
TIMEDELTA = 1e-01
OMEGA = 0.5  # Strength of Interaction

D = 30

p = Physics(dimension=D, interaction_strength=OMEGA, interaction_time=TIMEDELTA)

th = OMEGA * TIMEDELTA
alpha = complex(1/math.sqrt(2), 0)
beta = cmath.sqrt(1 - alpha**2)
phi = np.pi/2
# Phase shifts
delta1 = 0
delta2 = -phi

n1 = 1
n2 = 1

In [3]:
eta = use.create_ancilla_qobj(alpha, beta, phi)
rho1 = use.create_system_qobj('coherent', alpha=n1, n_dims=D)
rho2 = use.create_system_qobj('coherent', alpha=n2, n_dims=D)
rho = np.kron(rho1.full(), rho2.full())

use.Matrix(rho)

In [12]:
def dag(X: Qobj | np.ndarray):
    if isinstance(X, Qobj):
        return X.dag()
    elif isinstance(X, np.ndarray):
        return X.conj().T

def commutator(A: Qobj | np.ndarray, B: Qobj | np.ndarray, kind='regular'):
    if isinstance(A, Qobj) and isinstance(B, Qobj):
        if kind == 'regular':
            return A * B - B * A
        elif kind == 'anti':
            return A * B + B * A
    elif isinstance(A, np.ndarray) and isinstance(B, np.ndarray):
        if kind == 'regular':
            return A @ B - B @ A
        elif kind == 'anti':
            return A @ B + B @ A
    else:
        raise TypeError('A and B must be of the same type')


def dissipator(X: Qobj | np.ndarray, system: Qobj | np.ndarray, kind='regular'):
    sandwich = X @ system @ dag(X)
    if kind == 'anti':
        sandwich = dag(X) @ system @ X
    comm = commutator(dag(X) @ X, system, kind='anti')
    return sandwich - 1/2 * comm

def master_equation(system):
    ga = 2*alpha**2
    gb = beta**2*(1-np.cos(phi))
    # Bosonic Operators
    C = p.C.full()
    Cp = p.Cp.full()
    S = p.S.full()
    Sd = p.S.dag().full()
    first_line = 0.5*dissipator(np.kron(C, C) - 2*np.kron(S, Sd), system)
    first_line += dissipator(np.kron(C, S) + np.kron(S, Cp), system)
    second_line = 0.5*dissipator(np.kron(Cp, Cp) - 2*np.kron(Sd, S), system)
    second_line += dissipator(np.kron(Cp, Sd) + np.kron(Sd, C), system)
    return ga * first_line + gb * second_line

def evolve(system):
    delta_s = master_equation(system)
    return system + delta_s

In [13]:
def purity(rho):
    return np.trace(rho @ rho)

entropies = []
purities = []

In [14]:
# Evolve
for t in trange(TIMESTEPS):
    rho = evolve(rho)
    entropies.append(entropy_vn(rho))
    purities.append(purity(rho))
time = t + 1

  7%|▋         | 58/800 [34:48<7:25:16, 36.01s/it]  


KeyboardInterrupt: 

In [ ]:
# Calculate something
plt.plot(entropies, label='Entropy')
plt.plot(purities, label='Purity')
plt.legend()
plt.show()

In [8]:
use.Matrix(rho)